# Spectra geographically bins


In [1]:
import os

import numpy as np
import pandas as pd
import xarray as xr
import dask.dataframe as dd

# import cartopy.crs as ccrs
# import cartopy.feature as cfeature
# import geopandas as gpd
# from shapely.geometry import Polygon

%matplotlib inline
from matplotlib import pyplot as plt

import drifters.utils as ut
import pynsitu as pin
from GDP_lib import root_dir

In [2]:
from dask.distributed import Client

if True:
    from dask.distributed import Client
    from dask_jobqueue import PBSCluster

    # cluster = PBSCluster(cores=56, processes=28, walltime='04:00:00')
    # cluster = PBSCluster(cores=7, processes=7, walltime='04:00:00')
    cluster = PBSCluster(cores=3, processes=3, walltime="04:00:00")
    w = cluster.scale(jobs=5)
    # from dask_jobqueue import PBSCluster
    # cluster = PBSCluster()
    # w = cluster.scale(jobs=8) # 5 for gps, 8 for argos
else:
    from dask.distributed import LocalCluster

    cluster = LocalCluster()

client = Client(cluster)
client

/home1/datahome/mdemol/.miniconda3/envs/mdenv/lib/python3.9/site-packages/dask_jobqueue/core.py:237: FutureWarning: extra has been renamed to worker_extra_args. You are still using it (even if only set to []; please also check config files). If you did not set worker_extra_args yet, extra will be respected for now, but it will be removed in a future release. If you already set worker_extra_args, extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/home1/datahome/mdemol/.miniconda3/envs/mdenv/lib/python3.9/site-packages/dask_jobqueue/core.py:255: FutureWarning: job_extra has been renamed to job_extra_directives. You are still using it (even if only set to []; please also check config files). If you did not set job_extra_directives yet, job_extra will be respected for now, but it will be removed in a future release. If you already set job_extra_directives, job_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/home1/datahome/mdemol/.min

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.148.0.68:8787/status,
Dashboard: http://10.148.0.68:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.148.0.68:33571,Workers: 0
Dashboard: http://10.148.0.68:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


# Choose key 

In [3]:
key = "argos"  # 'gps' or 'argos'

# Load

In [4]:
# LOAD files with velocities and accelerations computed

parquet = os.path.join(root_dir, key + "_av_time.parquet")
# parquetd = os.path.join(root_dir, key + "_av_time_w.parquet")

# drop all variables but relevant ones
col = [
    "id",
    "time",
    "lon",
    "lat",
    "ve",
    "vn",
    "ae",
    "an",
    "vex",
    "vny",
    "aex",
    "any",
    "vex_diff",
    "vny_diff",
]
df = dd.read_parquet(parquet).reset_index()[col].persist()
# dfd = dd.read_parquet(parquetd).reset_index()[col].persist()

In [5]:
df

,id,time,lon,lat,ve,vn,ae,an,vex,vny,aex,any,vex_diff,vny_diff
npartitions=530,,,,,,,,,,,,,,
,int64,datetime64[ns],float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [6]:
len(df)

124338845

In [7]:
df.nunique().compute()

id              11073
time           269381
lon          29710957
lat          14029227
ve              43647
vn              42112
ae            1432538
an            1311017
vex         124327661
vny         124328113
aex         124316304
any         124316652
vex_diff    124338698
vny_diff    124338755
dtype: int64

In [8]:
nb_traj = df.id.nunique().compute()

_________________
# Generate Spectra

In [9]:
df = df.rename(columns={"time": "date"})
df["ven2"] = df.ve**2 + df.vn**2

# add time in hours
time_unit = pd.Timedelta("1H")
t_ref = pd.Timestamp(2000, 1, 1)
df["time"] = (df["date"] - t_ref) / time_unit

In [10]:
T_str = "60D"  # days
dt = "1H"  # sampling

N = int(pd.Timedelta(T_str) / pd.Timedelta(dt))  # output size
T = pd.Timedelta(T_str) / time_unit  # must be in the same units than time

columns = [
    "lon",
    "lat",
]
columns0 = columns + ["ve", "vn"]
columns1 = columns + ["ae", "an"]
columns2 = columns + ["vex", "vny"]
columns3 = columns + ["aex", "any"]
columns4 = columns + ["vex_diff", "vny_diff"]

labels = ["ven", "aen", "vxy", "axy", "vxydiff"]
Columns = {
    "ven": columns0,
    "aen": columns1,
    "vxy": columns2,
    "axy": columns3,
    "vxydiff": columns4,
}

In [11]:
def process_uv(lon, lat, u, v, N, dt, **kwargs):
    """Wraps spectral calculation: add complex velocity
    Assumes the time series is regularly sampled

    Parameters:
    -----------
        u, v: pd.Series
            zonal, meridional index by time (in days)
        N: int,
            length of the spectrum
        dt: float
            Time sampling in days
        **kwargs:
            passed to mit_equinox.drifters.get_spectrum
    """
    if lon is None:
        uv = None
    else:
        uv = u + 1j * v
    return pin.tseries.get_spectrum(uv, N, dt=dt, **kwargs)

In [12]:
df.head()

,id,date,lon,lat,ve,vn,ae,an,vex,vny,aex,any,vex_diff,vny_diff,ven2,time
0,8707978,1987-10-02 13:00:00,-137.744492,46.454159,0.3154,-0.0090,-1.333335e-06,-0.000013,0.400144,0.468731,-0.000015,-0.000081,0.400120,0.468670,0.099558,-107363.0
1,8707978,1987-10-02 14:00:00,-137.728226,46.459862,0.3094,-0.0900,-1.333335e-06,-0.000013,0.320565,0.029831,-0.000015,-0.000081,0.320577,0.029862,0.103828,-107362.0
2,8707978,1987-10-02 15:00:00,-137.714447,46.456089,0.3058,-0.0997,-5.833308e-07,-0.000002,0.299095,-0.108045,0.000003,0.000005,0.299126,-0.107957,0.103454,-107361.0
3,8707978,1987-10-02 16:00:00,-137.700195,46.452862,0.3052,-0.1041,-3.513888e-06,-0.000004,0.308067,-0.096442,0.000002,0.000002,0.308112,-0.096296,0.103984,-107360.0
4,8707978,1987-10-02 17:00:00,-137.685577,46.449841,0.2805,-0.1314,-4.027780e-06,-0.000012,0.286443,-0.199157,-0.000014,-0.000059,0.286582,-0.198973,0.095946,-107359.0


In [13]:
len(df)

124338845

### Compute spectra

In [14]:
# pin.drifters.
group = tuple(df.get_partition(0)["id"].loc[0].values.compute())[0]
dfg = df.groupby("id").get_group(group).compute()

out = pin.drifters.time_window_processing(
    dfg, process_uv, columns0, T, N, id_label="id", dt=dt, geo=True
)
# dfg.set_index("time").plot(x="lon", y="lat")


Df_chunked = {}
for l in Columns:
    df_chunked = (
        df.groupby("id")
        .apply(
            pin.drifters.time_window_processing,
            process_uv,
            Columns[l],
            T,
            N,
            id_label="id",
            dt=dt,
            geo=True,
            meta=out,
        )
        .persist()
    )
    # recompute date
    df_chunked["date"] = t_ref + df_chunked.index * time_unit
    # rename x/y
    df_chunked = df_chunked.rename(columns=dict(x="lon", y="lat"))
    # add nrj
    # df_chunked['ven2'] = df.ve**2 +df.vn**2

    Df_chunked[l] = df_chunked

In [15]:
Df_chunked["aen"].tail()

,lon,lat,id,0.0,0.016666666666666666,0.03333333333333333,0.05,0.06666666666666667,0.08333333333333333,0.1,...,-0.15,-0.13333333333333333,-0.11666666666666667,-0.1,-0.08333333333333333,-0.06666666666666667,-0.05,-0.03333333333333333,-0.016666666666666666,date
147487.0,-179.871005,-11.975244,145703.0,2.380645e-14,5.010007e-14,3.642163e-13,1.323019e-12,4.313937e-13,5.058799e-13,2.024908e-12,...,2.763542e-12,9.807117e-14,6.640284e-13,1.196700e-12,5.647899e-14,1.474547e-12,1.123894e-12,6.803424e-13,2.767707e-13,2016-10-28 07:00:00
148207.0,178.648856,-13.726922,145703.0,1.292187e-13,4.166144e-15,6.296172e-14,1.024656e-12,1.379822e-12,2.686008e-13,4.126079e-13,...,6.658919e-12,1.555552e-12,2.078073e-12,1.321306e-12,1.278577e-12,4.611099e-12,4.150278e-12,1.558539e-12,2.223496e-12,2016-11-27 07:00:00
148927.0,176.900652,-14.278134,145703.0,4.096352e-13,4.178954e-13,1.221878e-13,3.080400e-12,6.939296e-12,1.745103e-12,8.575256e-15,...,6.096989e-13,3.071620e-12,4.078496e-12,1.689709e-12,2.632645e-12,2.152330e-11,2.230381e-11,4.551371e-12,7.471569e-14,2016-12-27 07:00:00
149647.0,173.924280,-14.550017,145703.0,5.697027e-13,2.316566e-12,8.451948e-12,6.727220e-12,7.412221e-13,4.158147e-12,3.128578e-12,...,9.953363e-13,4.089224e-13,7.445718e-13,1.038335e-13,2.876857e-12,4.749523e-12,2.584859e-12,7.823834e-13,1.290914e-13,2017-01-26 07:00:00
150367.0,171.219387,-15.276158,145703.0,3.232104e-13,2.405287e-12,3.689545e-12,6.004295e-12,2.683437e-12,3.836132e-12,1.324006e-12,...,4.361785e-13,2.924997e-13,2.480702e-12,1.805236e-12,2.940023e-12,2.416977e-12,1.765523e-13,3.840724e-13,1.217311e-13,2017-02-25 07:00:00


In [16]:
did = Df_chunked["aen"].id.compute()

In [17]:
print(f"{did.nunique()} trajectories with more than T = 60 days among the {nb_traj}")

10624 trajectories with more than T = 60 days among the 11073


In [18]:
len(did)

159361

## Store spectra

In [19]:
DF = Df_chunked
DFs = {}
for l in DF:
    DFs[l] = DF[l].rename(
        columns={i: str(i) for i in list(DF[l].columns) if isinstance(i, float)}
    )

In [20]:
root_dir = "/home1/datawork/mdemol/GDP"


def store_spectra(Df, root_dir=root_dir):
    for var_key in Columns:
        parquet = os.path.join(root_dir, key + "_" + var_key + "_spectra.parquet")
        df = Df[var_key].repartition(partition_size="100MB").persist()
        df.to_parquet(parquet, engine="pyarrow")
        print(var_key)


store_spectra(DFs)

ven
aen
vxy
axy
vxydiff


---

# Geographically binned

https://github.com/apatlpo/mit_equinox/blob/master/parcels/spectra_binned.ipynb

In [23]:
DF = {}
labels = ["ven", "aen", "vxy", "axy", "vxydiff"]
for l in labels:
    parquet = os.path.join(root_dir, key + "_" + l + "_spectra.parquet")
    df = dd.read_parquet(parquet)
    df = df.rename(
        columns={
            i: float(i)
            for i in list(df.columns)
            if i not in ["lon", "lat", "id", "date"]
        }
    )
    DF[l] = df

In [24]:
DF["aen"].head()

,lon,lat,id,0.0,0.016666666666666666,0.03333333333333333,0.05,0.06666666666666667,0.08333333333333333,0.1,...,-0.15,-0.13333333333333333,-0.11666666666666667,-0.1,-0.08333333333333333,-0.06666666666666667,-0.05,-0.03333333333333333,-0.016666666666666666,date
15542.0,-12.543511,45.007263,28698.0,1.034071e-14,1.277673e-15,1.462905e-13,5.061532e-13,3.169983e-13,7.912768e-14,7.363554e-14,...,5.336356e-13,2.304986e-13,6.848399e-13,1.508994e-12,7.000057e-13,7.161600e-13,9.083949e-13,3.068311e-13,2.730558e-13,2001-10-09 14:00:00
16262.0,-11.524807,44.211897,28698.0,5.649764e-13,2.331408e-13,2.732889e-13,2.109914e-13,1.876818e-13,3.969372e-14,1.228001e-12,...,4.851697e-12,1.389345e-12,6.488099e-15,1.731002e-12,5.527162e-12,6.117597e-12,2.724244e-12,1.354692e-13,3.685465e-13,2001-11-08 14:00:00
16982.0,-13.137979,43.852229,28698.0,1.531180e-11,2.806829e-11,3.040208e-11,2.541344e-11,1.624577e-11,1.762816e-12,8.953112e-13,...,2.462381e-11,1.619803e-11,3.825376e-11,7.312631e-11,8.168940e-11,5.702906e-11,1.909632e-11,1.321521e-12,5.365345e-12,2001-12-08 14:00:00
19358.0,-164.644116,-47.286195,34363.0,7.647537e-14,2.240813e-13,3.203893e-13,1.066084e-12,2.470268e-12,1.152009e-12,1.247049e-13,...,8.182521e-13,1.604389e-13,4.227115e-14,2.215719e-13,8.046309e-13,9.306473e-13,2.023760e-13,4.914827e-14,7.029101e-14,2002-03-17 14:00:00
20078.0,-165.220319,-46.113563,34363.0,6.534643e-14,1.568091e-13,4.419407e-14,1.361704e-14,2.188226e-13,6.036946e-13,8.048099e-13,...,3.750019e-12,2.714033e-12,2.974435e-12,1.331309e-12,1.169046e-12,1.215799e-12,5.024046e-13,4.226592e-13,5.470840e-14,2002-04-16 14:00:00


In [25]:
# bin geographically
dl = 2
lon_bins = np.arange(
    -180.0, 180.0 + dl, dl
)  # CAUTION : add dl to upper bound (prevent from pb with last binning intervals)
lat_bins = np.arange(-90, 90 + dl, dl)

### Computing

In [26]:
DF_geo = {}
Ds = {}
for l in DF:
    # DF[l].drop(columns=['ven2'], axis=1, inplace=True)
    DF[l]["lon"] = (DF[l]["lon"] + 180) % 360 - 180
    DF[l]["lon_cut"] = DF[l]["lon"].map_partitions(pd.cut, bins=lon_bins)
    DF[l]["lat_cut"] = DF[l]["lat"].map_partitions(pd.cut, bins=lat_bins)
    DF_geo[l] = (
        DF[l].groupby(["lon_cut", "lat_cut"]).mean().compute()
    )  # MEAN SPECTRUM OVER ALL SEGMENT IN THE LON, LAT bins

    # Converting pandas dataframe in xarray dataset:
    index = pd.MultiIndex.from_arrays(
        [
            DF_geo[l].index.map(lambda v: v[0].mid),
            DF_geo[l].index.map(lambda v: v[1].mid),
        ],
        names=("lon_cut", "lat_cut"),
    )
    ds = (
        (
            pd.melt(
                DF_geo[l]
                .set_index(index)
                .drop(columns=["id", "lon", "lat"])
                .reset_index(),
                id_vars=[
                    "lon_cut",
                    "lat_cut",
                ],
                var_name="frequency",
            ).rename(
                columns={
                    "lon_cut": "lon_bins",
                    "lat_cut": "lat_bins",
                    "value": "E_" + l,
                }
            )
        )
        .to_xarray()
        .set_index(index=["lon_bins", "lat_bins", "frequency"])
        .unstack()
    )
    ds["frequency"] = ds["frequency"].astype(float)
    ds = ds.sortby("frequency")
    # ds = ds.chunk({'frequency': 100})
    Ds[l] = ds
    print(l)

# nb counts
dsc = (
    DF[l]
    .groupby(["lon_cut", "lat_cut"])
    .size()
    .compute()
    .to_frame("nb_geobins")
    .set_index(index)
    .reset_index()
    .to_xarray()
    .rename({"lon_cut": "lon_bins", "lat_cut": "lat_bins"})
    .set_index(index=["lon_bins", "lat_bins"])
    .unstack()
)

ven
aen
vxy
axy
vxydiff


In [27]:
ds = xr.merge(list(Ds.values()) + [dsc])

# set attrs
ds.E_ven.attrs = {
    "long_name": "Power density spectra v = ve + jvn",
    "units": r"$m^2/s^2/cpd$",
    "description": "LOWESS method",
}
ds.E_vxy.attrs = {
    "long_name": "Power density spectra v = vx + jvy",
    "units": r"$m^2/s^2/cpd$",
    "description": "geoid method",
}
ds.E_vxydiff.attrs = {
    "long_name": "Power density spectra v = vx + jvy",
    "units": r"$m^2/s^2/cpd$",
    "description": "finite differentiation method",
}
ds.E_axy.attrs = {
    "long_name": "Power density spectra a = ax + jay",
    "units": r"$m^2/s^4/cpd$",
    "description": "from position",
}
ds.E_aen.attrs = {
    "long_name": "Power density spectra a = ae + jan",
    "units": r"$m^2/s^4/cpd$",
    "description": "from ve,vn",
}

ds.frequency.attrs = {"long_name": "frequency", "units": "cpd"}
ds.nb_geobins.attrs = {
    "long_name": "nb_geobins",
    "description": "number of T days time window spectra per bins",
}

In [28]:
ds["E_venw2"] = ds["E_ven"] * (ds["frequency"] * 2 * np.pi / 3600 / 24) ** 2
ds["E_vxyw2"] = ds["E_vxy"] * (ds["frequency"] * 2 * np.pi / 3600 / 24) ** 2

In [29]:
ds.E_venw2.attrs = {
    "long_name": "E_ven $  \omega^2$",
    "units": r"$m^2/s^4/cpd$",
    "description": "from position",
}
ds.E_vxyw2.attrs = {
    "long_name": "E_vxy $  \omega^2$",
    "units": r"$m^2/s^4/cpd$",
    "description": "from ve,vn",
}

In [30]:
ds

<xarray.Dataset>
Dimensions:     (lon_bins: 180, lat_bins: 90, frequency: 1440)
Coordinates:
  * lon_bins    (lon_bins) float64 -179.0 -177.0 -175.0 ... 175.0 177.0 179.0
  * lat_bins    (lat_bins) float64 -89.0 -87.0 -85.0 -83.0 ... 85.0 87.0 89.0
  * frequency   (frequency) float64 -12.0 -11.98 -11.97 ... 11.95 11.97 11.98
Data variables:
    E_ven       (lon_bins, lat_bins, frequency) float64 nan nan nan ... nan nan
    E_aen       (lon_bins, lat_bins, frequency) float64 nan nan nan ... nan nan
    E_vxy       (lon_bins, lat_bins, frequency) float64 nan nan nan ... nan nan
    E_axy       (lon_bins, lat_bins, frequency) float64 nan nan nan ... nan nan
    E_vxydiff   (lon_bins, lat_bins, frequency) float64 nan nan nan ... nan nan
    nb_geobins  (lon_bins, lat_bins) int64 0 0 0 0 0 0 0 0 0 ... 0 1 0 0 1 0 0 0
    E_venw2     (lon_bins, lat_bins, frequency) float64 nan nan nan ... nan nan
    E_vxyw2     (lon_bins, lat_bins, frequency) float64 nan nan nan ... nan nan

In [31]:
ds.nb_geobins.sum()

<xarray.DataArray 'nb_geobins' ()>
array(159361)


## Store geo spectra

In [32]:
root_dir = "/home1/datawork/mdemol/GDP"
zarr = os.path.join(root_dir, key + f"_geospectra_{int(dl)}.zarr")

In [33]:
ds.to_zarr(zarr, mode="w")

In [34]:
ds_reload = xr.open_zarr(zarr).persist()
ds_reload

<xarray.Dataset>
Dimensions:     (lon_bins: 180, lat_bins: 90, frequency: 1440)
Coordinates:
  * frequency   (frequency) float64 -12.0 -11.98 -11.97 ... 11.95 11.97 11.98
  * lat_bins    (lat_bins) float64 -89.0 -87.0 -85.0 -83.0 ... 85.0 87.0 89.0
  * lon_bins    (lon_bins) float64 -179.0 -177.0 -175.0 ... 175.0 177.0 179.0
Data variables:
    E_aen       (lon_bins, lat_bins, frequency) float64 dask.array<chunksize=(23, 23, 360), meta=np.ndarray>
    E_axy       (lon_bins, lat_bins, frequency) float64 dask.array<chunksize=(23, 23, 360), meta=np.ndarray>
    E_ven       (lon_bins, lat_bins, frequency) float64 dask.array<chunksize=(23, 23, 360), meta=np.ndarray>
    E_venw2     (lon_bins, lat_bins, frequency) float64 dask.array<chunksize=(23, 23, 360), meta=np.ndarray>
    E_vxy       (lon_bins, lat_bins, frequency) float64 dask.array<chunksize=(23, 23, 360), meta=np.ndarray>
    E_vxydiff   (lon_bins, lat_bins, frequency) float64 dask.array<chunksize=(23, 23, 360), meta=np.ndarray>
    E_vxyw2     (lon_bins, lat_bins, frequency) float64 dask.array<chunksize=(23, 23, 360), meta=np.ndarray>
    nb_geobins  (lon_bins, lat_bins) int64 dask.array<chunksize=(180, 90), meta=np.ndarray>

In [36]:
cluster.close()